In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from tensorflow.python.keras.models import Sequential
# from tensorflow.python.keras.layers import LSTM, Dense

In [10]:
# Load and prepare your data
data = pd.read_csv('out.csv')
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)
data.drop(columns=['Unnamed: 0', 'id', 'screen'], inplace=True)
data['mood_shifted'] = data['mood'].shift(-1)
print(data.shape)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.fillna(data.mean()))
# Function to create sequences
def create_sequences(data, n_steps):
  data_X = np.delete(data, 3, axis=1)
  X, y = [], []
  for i in range(len(data)):
      end_ix = i + n_steps
      if end_ix > len(data):
          break
      seq_x, seq_y = data[i:end_ix], data[end_ix-1, 3]
      X.append(seq_x)
      y.append(seq_y)
  return np.array(X), np.array(y)


X,y = create_sequences(data_scaled, 7)


(1212, 17) (1212,)
[[0.4375     0.61538462 0.         ... 0.         0.0625     0.50526316]
 [0.55       0.44615385 0.0778501  ... 0.16666667 0.015625   0.53684211]
 [0.65       0.53846154 0.01813886 ... 0.06666667 0.         0.6       ]
 ...
 [0.5        0.75384615 0.06326391 ... 0.3        0.         0.63157895]
 [0.16666667 0.69230769 0.00992052 ... 0.13333333 0.         0.6       ]
 [0.3        0.26153846 0.07548952 ... 0.1        0.         0.63027294]]
[[0.4375     0.61538462 0.         ... 0.         0.0625     0.50526316]
 [0.55       0.44615385 0.0778501  ... 0.16666667 0.015625   0.53684211]
 [0.65       0.53846154 0.01813886 ... 0.06666667 0.         0.6       ]
 ...
 [0.5        0.75384615 0.06326391 ... 0.3        0.         0.63157895]
 [0.16666667 0.69230769 0.00992052 ... 0.13333333 0.         0.6       ]
 [0.3        0.26153846 0.07548952 ... 0.1        0.         0.63027294]]


In [ ]:
# Function to create sequences
def create_sequences(data_scaled, data_y n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data) - 1:
            break
        seq_x, seq_y = data[i:end_ix], data[end_ix, 2]  # Assuming 'mood' is at index 2
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:

# Define number of past days and create sequences
n_steps = 7
X, y = create_sequences(data_scaled, n_steps)
print(X,y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train, y_train)

# Build the LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(n_steps, 18)),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, verbose=1)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')


In [ ]:
import matplotlib.pyplot as plt

# Assuming `history` is the output from the `model.fit()` call
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)

# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Make predictions
predictions = model.predict(X_test)

# Plotting the actual vs predicted values
plt.figure(figsize=(10, 5))
plt.plot(y_test, label='Actual')
plt.plot(predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Mood')
plt.ylabel('Mood')
plt.xlabel('Test Sample')
plt.legend()
plt.show()